
#READ ME
Read these two articles if you are not familiar with Python:
1.   [Python Basics](https://medium.com/the-renaissance-developer/python-101-the-basics-441136fb7cc3)
2.   [Python Data Structures](https://medium.com/the-renaissance-developer/python-101-data-structures-a397bcc2bd30)
<br>


To Create Your Own Custom function you need to define a function in the form like this:

```
def function_name(my_moves, other_moves, current_round):
    return True # This means the function will make the player rat.
    # Code after this will NOT run
```


```
def another_function(my_moves, other_moves, current_round):
    if current_round == 10:
      return True #This means that if the currend round equals 10, this player  will rat
      # Code here will not run
     
     #However if the above condition is not met, it will not run the code in the if statement and continue below
     return False
     # Code here will not run
```



Every function will have access to 3 variables: <br>


1.  `my_moves` is a list of moves this player has made. This list is updated after every round. Ex: `[False, True, True, True]`   (This means the player has stayed silent on the first round and ratted every round after)<br>
2. `other_moves` is a list of moves the other player has made. This list is updated after every round. Ex: `[True, False, False, False]` (This means the other player ratted on the first round and stayed silent every round after)
3. `current_round` is an integer representing the number of rounds already played. Ex: `25` or `0`




IMPORTANT INDEXING NOTE: Python starts counting from 0. That means the first round will be the 0th round. To access the first round's decision, you must write `my_moves[0]` or `other_moves[0]` NOT `my_moves[1]` or `other_moves[1]`

In [ ]:
from functools import total_ordering
# @title Simulation Code (check it out if you want to, but make sure to run it by hitting the play button)


# Do not touch anything here (ignore this cell)
import random
import pandas as pd
import time
import json
import numpy as np
from IPython.display import display

def play_match(player1, player2, blindness, payoff_dict):
    total = 0
    player1TrueMoves = []
    player2TrueMoves = []
    player1ObsMoves = []
    player2ObsMoves = []
    player1currentreturnedmoves = []
    player2currentreturnedmoves = []
    results = {}
    results["score"] = [0, 0]
    invalidGame = False

    for i in range(rounds):
        # get moves, make sure they are booleans or lists of booleans
        if player1currentreturnedmoves:
            player1move = player1currentreturnedmoves.pop(0)
        else:
            player1move = player1(
                player1TrueMoves.copy(), player2ObsMoves.copy(), i
            )

        if player2currentreturnedmoves:
            player2move = player2currentreturnedmoves.pop(0)
        else:
            player2move = player2(
                player2TrueMoves.copy(), player1ObsMoves.copy(), i
            )

        if not (type(player1move) is bool or (type(player1move) is list and len(player1move) > 0 and type(player1move[0]) is bool)):
            print(f"On round {i} {player1.__name__} returned {player1move}, which is not allowed")
            invalidGame = True
            break

        if not (type(player2move) is bool or (type(player2move) is list and len(player2move) > 0 and type(player2move[0]) is bool)):
            print(f"On round {i} {player2.__name__} returned {player2move}, which is not allowed")
            invalidGame = True
            break

        if type(player1move) is list:
            player1currentreturnedmoves = player1move.copy()
            player1move = player1currentreturnedmoves.pop(0)

        if type(player2move) is list:
            player2currentreturnedmoves = player2move.copy()
            player2move = player2currentreturnedmoves.pop(0)

        player1ObsMoves.append(not player1move if (blindness[0] != 0 and random.random() < blindness[0]) else player1move)
        player1TrueMoves.append(player1move)
        player2ObsMoves.append(not player2move if (blindness[1] != 0 and random.random() < blindness[1]) else player2move)
        player2TrueMoves.append(player2move)

    # replay through game and score
    if not invalidGame:
        for i in range(rounds):
            # payoff matrix
            results["score"][0] += payoff_dict[(player1TrueMoves[i], player2TrueMoves[i])]
            results["score"][1] += payoff_dict[(player2TrueMoves[i], player1TrueMoves[i])]

            results["details"] = [player1TrueMoves, player2TrueMoves]
        total += results["score"][1]
    else:
        results["score"] = [None, None]
    return total, results


def run_no_noise_tournament(strats, rounds, payoff, seed=None):
    payoff_dict = {
        (False, False): payoff[0],
        (True, False): payoff[1],
        (False, True): payoff[2],
        (True, True): payoff[3]
    }

    data = {}
    for player1 in strats:
        data_player1 = {}
        player1_total = 0
        for player2 in strats:
            if seed is not None:
                np.random.seed(seed)
                random.seed(seed)
            total, results = play_match(player1, player2, [0, 0], payoff_dict)
            data_player1[player2.__name__] = results
            player1_total += total

        data_player1["Average"] = round(player1_total / len(strats), 2)
        data_player1["Total"] = player1_total
        data[player1.__name__] = data_player1
    return data


def run_noise_tournament(strats, rounds, blindness, num_rounds_to_avg, payoff, seeds=None):
    payoff_dict = {
        (False, False): payoff[0],
        (True, False): payoff[1],
        (False, True): payoff[2],
        (True, True): payoff[3]
    }

    data = {}
    for player1 in strats:
        data_player1 = {}
        player1_total = 0
        for player2 in strats:
            total_over_rounds = 0
            data_player1[player2.__name__] = {}
            data_player1[player2.__name__]["score"] = [0, 0]
            data_player1[player2.__name__]["details"] = []


            for i in range(num_rounds_to_avg):
                if seeds is not None:
                    np.random.seed(seeds[i])
                    random.seed(seeds[i])
                total, results = play_match(player1, player2, blindness, payoff_dict)
                total_over_rounds += total
                data_player1[player2.__name__]["score"][0] += results["score"][0]
                data_player1[player2.__name__]["score"][1] += results["score"][1]
                data_player1[player2.__name__]["details"].append(results["details"])
            data_player1[player2.__name__]["score"][0] /= num_rounds_to_avg
            data_player1[player2.__name__]["score"][1] /= num_rounds_to_avg
            player1_total += total_over_rounds


        data_player1["Average"] = round(player1_total / len(strats) / num_rounds_to_avg, 2)
        data_player1["Total"] = player1_total
        data[player1.__name__] = data_player1
    return data

def display_results(data):
    clean = {}
    for strategy1 in data:
        clean[strategy1] = {}
        for strategy2 in data[strategy1]:
            if isinstance(data[strategy1][strategy2], dict):
                clean[strategy1][strategy2] = data[strategy1][strategy2]['score']
            else:
                clean[strategy1][strategy2] = data[strategy1][strategy2]

    df = pd.DataFrame(data=clean).T.sort_index(axis=1)
    display(df)

def display_matchup_observed_history(data, player1, player2, complete_history=False, i=None):
    print(f"results for {player1.__name__} vs {player2.__name__}:")
    print(f"score: {data[player1.__name__][player2.__name__]['score']}")
    if i is None:
        details = data[player1.__name__][player2.__name__]['details']
    else:
        details = data[player1.__name__][player2.__name__]['details'][i]

    player1_moves = details[0]
    player2_moves = details[1]

    moves_df = pd.DataFrame({
            f'{player1.__name__}': player1_moves,
            f'{player2.__name__}': player2_moves
        })
    if complete_history:
        with pd.option_context('display.max_rows', None):
            display(moves_df)
    else:
        display(moves_df)


def test_function(function):
    test_cases = [[[True] * i, [False] * i, i] for i in range(0, 300)]

    has_error = False
    for test_case in test_cases:
        try:
            my_moves = test_case[0]
            my_moves_copy = my_moves.copy()
            other_moves = test_case[1]
            other_moves_copy = other_moves.copy()
            output = function(my_moves_copy, other_moves_copy, test_case[2])
            if my_moves != my_moves_copy or other_moves != other_moves_copy:
                print(f"On round {test_case[2]} your function {function.__name__} modified the input arrays, which is not allowed")
                return
                # for anyone snooping around in the code: this is not how
                # the actual simulation will be run, the functions will not
                # be checked for if it modifies the input arrays, we will
                # just pass in copies of the arrays
            if not (output is True or output is False or (type(output) is list and len(output) > 0) and type(output[0]) is bool ):
                print(f"On round {test_case[2]} your function {function.__name__} returned {output}, which is not True or False or a list of bools")
                return
        except Exception as e:
            print(f"ERROR\nOn round {test_case[2]} your function {function.__name__} produced the following error:\n{e}")
            has_error = True
    if not has_error:
        print("Your function works :)")

In [ ]:
# @title Default Functions (run them)

import random


def rat(my_moves, other_moves, current_round):
    # Always Rats (returns True)
    return True


def silent(my_moves, other_moves, current_round):
    # Always stays silent (returns False)
    return False


def rand(my_moves, other_moves, current_round):
    random_number = random.random()
    if random_number < 0.5:
        return True
    else:
        return False


def kinda_random(my_moves, other_moves, current_round):
    cheat_probability = 0.9
    if random.random() < cheat_probability:
        return True
    return False


def tit_for_tat(my_moves, other_moves, current_round):
    if len(other_moves) == 0:
        return False
    if other_moves[-1]:
        return True
    return False


def tit_for_two_tats(my_moves, other_moves, current_round):
    if len(other_moves) < 2:
        return False
    return other_moves[-1] and other_moves[-2]


def nuke_for_tat(my_moves, other_moves, current_round):
    if len(other_moves) == 0:
        return False
    if other_moves[-1] == True:
        return [True] * 200
    return False


def nuke_for_two_tats(my_moves, other_moves, current_round):
    if len(other_moves) < 2:
        return False
    indices = [i for i, x in enumerate(other_moves) if x]
    for i in range(len(indices) - 1):
        if indices[i] == indices[i + 1] - 1:
            return True
    return False


def nuke_for_five_tats(my_moves, other_moves, current_round):
    # Stays silent until the other player rats five times. If the other player's rats 5 times this player rats forever.
    if len(other_moves) == 0:
        return False
    num_of_betray = 0
    for i in other_moves:
        if i:
            num_of_betray += 1
    if (
        num_of_betray > 4
    ):  # you can change the 4 here to make this a nukeForXtats (remember that the number here should be one less than the number of tats)
        return [True] * 200
    return False


def two_tits_for_tat(my_moves, other_moves, current_round):
    if current_round == 0:
        return False
    if (current_round >= 1 and other_moves[-1]) or (
        current_round >= 2 and other_moves[-2]
    ):
        return True
    return False


def get_angry_after_twenty(my_moves, other_moves, current_round):
    # Cooperate for 20 rounds, then if opponent ever defected, defect forever
    if current_round < 20:
        return False
    else:
        if True in other_moves:  # if opponent ever defected
            return [True] * 200
        else:
            return False


def cooperate_at_multiples_of_three(my_moves, other_moves, current_round):
    # Cooperate only on rounds that are multiples of 3
    return [True, True, False] * 67


def alternate_every_five(my_moves, other_moves, current_round):
    # Every 5 rounds, switch between cooperate and defect
    return ([True] * 5 + [False] * 5) * 20


def suspicious_tit_for_tat(my_moves, other_moves, current_round):
    if current_round == 0:
        return True
    return other_moves[-1]

## add your own function(s) below

In [ ]:
def your_function(my_moves, other_moves, current_round):
  return True

## testing
To make sure your function runs correctly, replace `tit_for_tat` with the name of your function. Then, run the following code block.



In [ ]:
# Run this cell to test your code
test_function(tit_for_tat)

Your function works :)


## Inputs for the `main` function

1. `rounds` is the number of rounds that will be played. Remember that the time it takes to calculate n rounds is proportional to n^2 (it takes about 140 seconds to calculate 10000 rounds with 13 strategies). Try starting with 150 rounds.

2. `blindness` is a variable adjusts the percentage chance that player 1 and player 2's actions will misinterpreted. For example `[0.2, 0.5]` means player 1's actions will bee misinterpreted 20% of the time and player 2's actions will misinterpreted 50% of the time. For **no noise** this should be `[0.0, 0.0]` and for **with noise** this should be `[0.1, 0.1]`.

3. `funcs` is a list of functions that we will want to analyze. When adding custom functions, you must type in the EXACT name of the function.

4. `payoff` is a dictionary with tuples of two booleans as  of four numbers that set the payoffs for each outcome. The variable is organized like so: [both cooperate, you rat opponent cooperates, you cooperate opponent rats, both rat]. This should be `[5,9,0,1]`.

There are two other variables that you don't need to worry about but you can if you want to.

1. `seed` is an integer that acts as the random seed for any functions using the `random` or `numpy.random` modules, as well as for the noise simulation. This is useful for when you want to run noise tournaments multiple times but hold randomness constant. Defaults to `None`, meaning no seed is being used.

## Run Without Noise

In [ ]:
# input variables
funcs = [
    rat,
    silent,
    rand,
    kinda_random,
    tit_for_tat,
    tit_for_two_tats,
    nuke_for_tat,
    nuke_for_two_tats,
    nuke_for_five_tats,
    two_tits_for_tat,
    get_angry_after_twenty,
    cooperate_at_multiples_of_three,
    alternate_every_five,
    suspicious_tit_for_tat
]
rounds = 150
payoff = [5, 9, 0, 1]
seed = 42

# runs and formats the results
start = time.time()
data = run_no_noise_tournament(funcs, rounds, payoff, seed=seed)

print(f"{time.time() - start} seconds to calculate")

display_results(data)

0.04707694053649902 seconds to calculate


,Average,Total,alternate_every_five,cooperate_at_multiples_of_three,get_angry_after_twenty,kinda_random,nuke_for_five_tats,nuke_for_tat,nuke_for_two_tats,rand,rat,silent,suspicious_tit_for_tat,tit_for_tat,tit_for_two_tats,two_tits_for_tat
rat,121.71,1704,"[750, 75]","[550, 100]","[310, 130]","[254, 137]","[190, 145]","[158, 149]","[166, 148]","[758, 74]","[150, 150]","[1350, 0]","[150, 150]","[158, 149]","[166, 148]","[158, 149]"
silent,903.43,12648,"[375, 1050]","[250, 1150]","[750, 750]","[65, 1298]","[750, 750]","[750, 750]","[750, 750]","[380, 1046]","[0, 1350]","[750, 750]","[745, 754]","[750, 750]","[750, 750]","[750, 750]"
rand,623.14,8724,"[569, 578]","[396, 630]","[207, 711]","[199, 676]","[134, 737]","[87, 753]","[95, 752]","[577, 514]","[74, 758]","[1046, 380]","[556, 556]","[561, 552]","[791, 476]","[363, 651]"
kinda_random,215.43,3016,"[719, 161]","[525, 192]","[297, 234]","[270, 243]","[177, 249]","[145, 253]","[153, 252]","[723, 156]","[137, 254]","[1298, 65]","[241, 241]","[249, 240]","[358, 223]","[145, 253]"
tit_for_tat,617.0,8638,"[495, 495]","[500, 500]","[750, 750]","[240, 249]","[750, 750]","[750, 750]","[750, 750]","[552, 561]","[149, 158]","[750, 750]","[675, 675]","[750, 750]","[750, 750]","[750, 750]"
tit_for_two_tats,684.57,9584,"[480, 615]","[450, 900]","[750, 750]","[223, 358]","[750, 750]","[750, 750]","[750, 750]","[476, 791]","[148, 166]","[750, 750]","[745, 754]","[750, 750]","[750, 750]","[750, 750]"
nuke_for_tat,481.29,6738,"[749, 83]","[549, 108]","[750, 750]","[253, 145]","[750, 750]","[750, 750]","[750, 750]","[753, 87]","[149, 158]","[750, 750]","[157, 157]","[750, 750]","[750, 750]","[750, 750]"
nuke_for_two_tats,526.79,7375,"[748, 91]","[548, 116]","[750, 750]","[252, 153]","[750, 750]","[750, 750]","[750, 750]","[752, 95]","[148, 166]","[750, 750]","[745, 754]","[750, 750]","[750, 750]","[750, 750]"
nuke_for_five_tats,537.14,7520,"[745, 115]","[537, 150]","[750, 750]","[249, 177]","[750, 750]","[750, 750]","[750, 750]","[737, 134]","[145, 190]","[750, 750]","[745, 754]","[750, 750]","[750, 750]","[750, 750]"
two_tits_for_tat,525.07,7351,"[555, 420]","[549, 108]","[750, 750]","[253, 145]","[750, 750]","[750, 750]","[750, 750]","[651, 363]","[149, 158]","[750, 750]","[157, 157]","[750, 750]","[750, 750]","[750, 750]"


In [ ]:
#This lets you analyze the precise decisions of the two players.
display_matchup_observed_history(data, nuke_for_five_tats, alternate_every_five)

results for nuke_for_five_tats vs alternate_every_five:
score: [745, 115]


,nuke_for_five_tats,alternate_every_five
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
...,...,...
145,True,False
146,True,False
147,True,False
148,True,False


In [ ]:
# if you want to see the full results, use the "complete_history=True" parameter:
display_matchup_observed_history(data, nuke_for_five_tats, alternate_every_five, complete_history=True)

results for nuke_for_five_tats vs alternate_every_five:
score: [745, 115]


,nuke_for_five_tats,alternate_every_five
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
5,True,False
6,True,False
7,True,False
8,True,False
9,True,False


## Run With Noise

In [ ]:
# input variables
funcs = [rat, silent, rand, kinda_random, tit_for_tat, tit_for_two_tats, nuke_for_tat, nuke_for_two_tats, nuke_for_five_tats, get_angry_after_twenty, cooperate_at_multiples_of_three, alternate_every_five, suspicious_tit_for_tat]
rounds = 150
blindness = [0.1, 0.1]
num_rounds_to_avg = 2
payoff = [5, 9, 0, 1]
seeds = [42, 43]

# runs and formats the results
start = time.time()
data = run_noise_tournament(funcs, rounds, blindness, num_rounds_to_avg, payoff, seeds=seeds)

print(f"{time.time() - start} seconds to calculate")

display_results(data)

In [ ]:
# for noise games, there are multiple observed histories

display_matchup_observed_history(data, nuke_for_five_tats, alternate_every_five)

results for nuke_for_five_tats vs alternate_every_five:
score: [734.0, 135.5]


,nuke_for_five_tats,alternate_every_five
0,"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, True, True..."
1,"[True, True, True, True, True, False, False, F...","[True, True, True, True, True, False, False, F..."


In [ ]:
# to choose a specific one, specify the 'i' parameter to be the index that you want

display_matchup_observed_history(data, nuke_for_five_tats, alternate_every_five, i=1, complete_history=True)

results for nuke_for_five_tats vs alternate_every_five:
score: [734.0, 135.5]


,nuke_for_five_tats,alternate_every_five
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
5,True,False
6,True,False
7,True,False
8,True,False
9,True,False
